In [ ]:
# only run if on a multi-GPU setup
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "3"

In [ ]:
import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt

from utils.data import tfr_dataset_eager, parse_img_label_tfr

In [ ]:
# can easily switch to other datasets by changing the tfrecord path
# make sure to use the correct shape in the parse function
# e.g. 32,32,3 for color images and CNNs.
parse_fn = lambda x: parse_img_label_tfr(x, (1024,))
mnist_data = tfr_dataset_eager(["/cache/tfrs/mnist_train.tfr"], 256, parse_fn, shufrep=60000)
mnist_test = tfr_dataset_eager(["/cache/tfrs/mnist_test.tfr"], 1000, parse_fn)

# if you wanna try SVHN you need this because for some reason the digit 0 has index 10. this replaces it by 0.
#mnist_data = mnist_data.map(lambda x, lbls: (x, tf.where(tf.equal(lbls, 10), tf.zeros(tf.shape(lbls), tf.int32), lbls)))
#mnist_test = mnist_test.map(lambda x, lbls: (x, tf.where(tf.equal(lbls, 10), tf.zeros(tf.shape(lbls), tf.int32), lbls)))


In [ ]:
# look at some data points
for img_batch, lbl_batch in mnist_data:
    plt.figure(figsize=(1,1))
    plt.imshow(np.reshape(img_batch[0], [32, 32]), cmap="Greys_r")
    plt.show()
    print(lbl_batch[0])
    input()

In [ ]:
# basic linear model. also displays tensorboard usage
w = tf.Variable(tf.random.uniform([1024, 10], -.1, .1))
b = tf.Variable(tf.zeros(10))
loss_fn = tf.losses.SparseCategoricalCrossentropy(from_logits=True)
opt = tf.optimizers.Adam()

with tf.summary.create_file_writer("/cache/tensorboard-logdir/linear").as_default():
    for step, (img_batch, lbl_batch) in enumerate(mnist_data):
        if step > 1500:
            break
        if not step % 100:
            print("Step", step)
        with tf.GradientTape() as tape:
            logits = tf.matmul(img_batch, w) + b
            xent = loss_fn(lbl_batch, logits)
        grads = tape.gradient(xent, [w, b])
        opt.apply_gradients(zip(grads, [w, b]))
        tf.summary.scalar("cross entropy", xent, step=step)

In [ ]:
# keras metrics for easy evaluation etc. (could also include these during training!)
val_acc_metric = tf.keras.metrics.SparseCategoricalAccuracy()
for (img_batch, lbl_batch) in mnist_test:
    logits = tf.matmul(img_batch, w) + b
    val_acc_metric(lbl_batch, logits)
val_acc = val_acc_metric.result()
print(val_acc)

In [ ]:
# try CIFAR10, CNN, with keras.Sequential
parse_fn = lambda x: parse_img_label_tfr(x, (32, 32, 3))
cifar_data = tfr_dataset_eager(["/cache/tfrs/cifar10_train.tfr"], 256, parse_fn, shufrep=60000)
cifar_test = tfr_dataset_eager(["/cache/tfrs/cifar10_test.tfr"], 1000, parse_fn)

model = tf.keras.Sequential([tf.keras.layers.Conv2D(16, 3, activation=tf.nn.relu, padding="same"),
                             tf.keras.layers.MaxPooling2D(padding="same"),
                             tf.keras.layers.Conv2D(32, 3, activation=tf.nn.relu, padding="same"),
                             tf.keras.layers.MaxPooling2D(padding="same"),
                             tf.keras.layers.Conv2D(64, 3, activation=tf.nn.relu, padding="same"),
                             tf.keras.layers.MaxPooling2D(padding="same"),
                             tf.keras.layers.Flatten(),
                             tf.keras.layers.Dense(10)])

loss_fn = tf.losses.SparseCategoricalCrossentropy(from_logits=True)
opt = tf.optimizers.Adam()

# optional to build now, but only after building do we have variables available
model.build((None, 32, 32, 3))
varis = model.trainable_variables


# this decorator activates graph mode for this function (and everything it calls)
# try removing it and see how many smiley faces are printed in each case
@tf.function
def train(imgs, lbls):
    print(":-)")
    with tf.GradientTape() as tape:
        logits = model(imgs)
        xent = loss_fn(lbls, logits)
    grads = tape.gradient(xent, varis)
    opt.apply_gradients(zip(grads, varis))
    
    return xent


# in principle the whole training loop should be wrappable in a tf.function
# but I haven't figured it out yet :p
for step, (img_batch, lbl_batch) in enumerate(cifar_data):
    if step > 5000:
        break
        
    xent = train(img_batch, lbl_batch)
        
    if not step % 100:
        print("Step", step)
        print("loss", xent)

In [ ]:
# comparing keras metric to manual computation
# note that this is eager again
val_acc_metric = tf.keras.metrics.SparseCategoricalAccuracy()

correct = 0
total = 0
for (img_batch, lbl_batch) in cifar_test:
    logits = model(img_batch)
    val_acc_metric(lbl_batch, logits)
    
    preds = logits.numpy().argmax(axis=-1)
    total += len(lbl_batch.numpy())
    correct += sum(preds == lbl_batch.numpy())


print(val_acc_metric.result())
print(correct/total)

In [ ]:
# broken example. bonus assignment: make it work (on the GPU) ;-)
from tensorflow.python.ops import control_flow_util
control_flow_util.ENABLE_CONTROL_FLOW_V2 = True

model = tf.keras.Sequential([tf.keras.layers.Conv2D(16, 3, activation=tf.nn.relu, padding="same"),
                             tf.keras.layers.MaxPooling2D(padding="same"),
                             tf.keras.layers.Conv2D(32, 3, activation=tf.nn.relu, padding="same"),
                             tf.keras.layers.MaxPooling2D(padding="same"),
                             tf.keras.layers.Conv2D(64, 3, activation=tf.nn.relu, padding="same"),
                             tf.keras.layers.MaxPooling2D(padding="same"),
                             tf.keras.layers.Flatten(),
                             tf.keras.layers.Dense(10)])

loss_fn = tf.losses.SparseCategoricalCrossentropy(from_logits=True)
opt = tf.optimizers.Adam()

model.build((None, 32, 32, 3))
varis = model.trainable_variables


def train_step(imgs, lbls):
    lbls = tf.where(tf.equal(lbls, 10), tf.zeros(tf.shape(lbls), tf.int32), lbls)
    with tf.GradientTape() as tape:
        logits = model(imgs)
        xent = loss_fn(lbls, logits)
    grads = tape.gradient(xent, varis)
    opt.apply_gradients(zip(grads, varis))
    
    return xent


@tf.function
def train_loop():
    step = tf.constant(0, dtype=tf.int32)
    
    for img_batch, lbl_batch in cifar1500:
        xent = train_step(img_batch, lbl_batch)
        step += 1
        
        if tf.equal(step % 100, 0):
            tf.print("Step", step)
            tf.print("loss", xent)
    

cifar1500 = cifar_data.take(1500)
train_loop()        

In [ ]:
# evaluate yet again
val_acc_metric = tf.keras.metrics.SparseCategoricalAccuracy()
for (img_batch, lbl_batch) in cifar_test:
    logits = model(img_batch)
    val_acc_metric(lbl_batch, logits)
val_acc = val_acc_metric.result()
print(val_acc)

In [ ]:
# we can use the full keras capacities for easy training (but harder to customize)
model = tf.keras.Sequential([tf.keras.layers.Conv2D(16, 3, activation=tf.nn.relu, padding="same"),
                             tf.keras.layers.MaxPooling2D(padding="same"),
                             tf.keras.layers.Conv2D(32, 3, activation=tf.nn.relu, padding="same"),
                             tf.keras.layers.MaxPooling2D(padding="same"),
                             tf.keras.layers.Conv2D(64, 3, activation=tf.nn.relu, padding="same"),
                             tf.keras.layers.MaxPooling2D(padding="same"),
                             tf.keras.layers.Flatten(),
                             tf.keras.layers.Dense(10)])

model.compile(optimizer=tf.optimizers.Adam(),
              loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

model.fit(cifar_data, epochs=10, steps_per_epoch=150)

In [ ]:
# evaluation is easy! uses the metrics specified in compile + loss
model.evaluate(cifar_test)
# predict on whole test set at once, returns a numpy array with logits
preds = model.predict(cifar_test)

In [ ]:
# we can still use the model as callable!
model(tf.random.normal([10, 32, 32, 3]))